# Submitted by: N.PAVAN PRATEEK


# For Bipolar Factory internship Assessment.


# Date of Submission: 27rd April 2020

Crawl news & information websites & anticipate the likelihood of its virality

In [6]:
pip install newspaper3k

Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
from bs4 import BeautifulSoup
from newspaper import Article  
import csv 
import nltk
import pandas as pd
import numpy as np

# Crawling News from Times  Website

In [2]:
url = "https://timesofindia.indiatimes.com/world"
r = requests.get(url)

soup = BeautifulSoup(r.content, 'html5lib')
table = soup.findAll('a', attrs = {'class':'w_img'})

In [3]:
news=[]
for row in table: 
    if not row['href'].startswith('http'):
        news.append('https://timesofindia.indiatimes.com'+row['href'])

In [21]:
df=[]
for i in news:
    ap= Article(i, language="en")
    ap.download() 
    ap.parse() 
    ap.nlp() 
    d={}
    d['heading']=ap.title
    d['content']=ap.text
    d['brief']=ap.summary
    d['main']=ap.keywords
    df.append(d)

In [22]:
data=pd.DataFrame(df)
data.head()

,heading,content,brief,main
0,Brazil's Supreme Court authorizes investigatio...,Brazil's President Jair Bolsonaro (AP)\n\nDown...,Brazil's President Jair Bolsonaro (AP)Download...,"[court, bolsonaro, times, federal, supreme, au..."
1,Covid-19: Mayor entertains residents after imp...,"Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...","Apr 27, 2020, 08:27AM ISTSource: APThe videos,...","[entertains, curfew, residents, imposing, dont..."
2,UK PM Boris Johnson returns to work after reco...,"Apr 27, 2020, 08:40AM IST\n\nSource: Times Now...","Apr 27, 2020, 08:40AM ISTSource: Times NowBrit...","[street, times, uk, virus, boris, worst, retur..."
3,"1 officer dead, 1 wounded in Louisiana shooting","Apr 27, 2020, 08:28AM IST\n\nSource: AP\n\nPol...","Apr 27, 2020, 08:28AM ISTSource: APPolice say ...","[say, officers, paul, wounded, officer, dead, ..."
4,Covid-19: Lockdown reveals fresh and clean air...,"Apr 22, 2020, 04:03PM IST\n\nSource: AP\n\nInd...","Apr 22, 2020, 04:03PM ISTSource: APIndia's ext...","[lockdown, clean, air, fresh, unprecedented, t..."


# Model for predicting virality of news

In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [24]:
FILEPATH=r"C:\Users\pavan\Downloads\crawl.csv"

In [25]:
def clean(data):
    cleans= {x: x.lower().strip() for x in list(data)}
    return data.rename(index=str, columns=cleans)

def TrainTestSplit(X, Y, R=0, test_size=0.2):
    return train_test_split(X, Y, test_size=test_size, random_state=R)

#### I have droped these features 

In [26]:
full= clean(pd.read_csv(FILEPATH))
train_set, test_set = train_test_split(full_data, test_size=0.20, random_state=42)

x_train = train_set.drop(['url','shares','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','abs_title_subjectivity','abs_title_sentiment_polarity'], axis=1)
y_train = train_set['shares']

x_test = test_set.drop(['url','shares', 'self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess','abs_title_subjectivity','abs_title_sentiment_polarity'], axis=1)
y_test = test_set['shares']

In [27]:
clf = RandomForestRegressor(random_state=42)
clf.fit(x_train, y_train)

C:\Users\pavan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

# Converting Crawled News

In [44]:
from nltk.tokenize import word_tokenize 
from textblob import TextBlob
def kill(text):
    text=text
    return word_tokenize(text)

In [45]:
df2=[]
for i in news:
    pred_info={}
    p= Article(i, language="en")
    p.download() 
    p.parse()
    a2=TextBlob(p.text)
    polarity=a2.sentiment.polarity
    title=TextBlob(p.title)
    pred_info['text']=p.text
    pred_info['n-title']=len(kill(p.title))
    pred_info['n-content']=len(kill(p.text))
    pred_info['n-hrefs']=p.html.count("https://timesofindia.indiatimes.com")
    pred_info['n-imgs']=len(p.images)
    pred_info['n-subjectivity']=title.sentiment.subjectivity
    pred_info['n-sentiment_polarity']=title.sentiment.polarity
    df2.append(pred_info)

In [42]:
pr=pd.DataFrame(df2)
pt=pr.drop(['text'],axis=1)
pr.head()

,text,n-title,n-content,n-hrefs,n-imgs,n-subjectivity,n-sentiment_polarity
0,Brazil's President Jair Bolsonaro (AP)\n\nDown...,13,384,229,13,0.0,0.000000
1,"Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...",8,190,185,9,0.0,0.000000
2,"Apr 27, 2020, 08:40AM IST\n\nSource: Times Now...",11,97,185,9,0.0,0.000000
3,"Apr 27, 2020, 08:28AM IST\n\nSource: AP\n\nPol...",9,153,185,9,0.4,-0.200000
4,"Apr 22, 2020, 04:03PM IST\n\nSource: AP\n\nInd...",10,124,185,9,0.6,0.333333


# Final Results of the Likelihood of Virality of News

In [43]:
test2=pd.DataFrame(clf.predict(pt),pr['text'])
test2.reset_index(level=0, inplace=True)
test2 = test2.rename(index=str, columns={"index": "News", 0: "Virality"})
test2

,text,Virality
0,Brazil's President Jair Bolsonaro (AP)\n\nDown...,19131.8
1,"Apr 27, 2020, 08:27AM IST\n\nSource: AP\n\nThe...",5050.0
2,"Apr 27, 2020, 08:40AM IST\n\nSource: Times Now...",5456.0
3,"Apr 27, 2020, 08:28AM IST\n\nSource: AP\n\nPol...",6739.6
4,"Apr 22, 2020, 04:03PM IST\n\nSource: AP\n\nInd...",6770.0
5,"Apr 22, 2020, 03:58PM IST\n\nSource: AP\n\nIra...",1877.1
6,"Apr 22, 2020, 04:04PM IST\n\nSource: AP\n\nAn ...",8010.0
7,"Apr 22, 2020, 03:57PM IST\n\nSource: AP\n\nFou...",4726.0
8,"Apr 21, 2020, 09:25AM IST\n\nSource: Times Now...",36140.0
9,"Apr 21, 2020, 03:38PM IST\n\nSource: AP\n\nA s...",6135.1
